<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Account-Endpoints/blob/main/Query_position_data_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Query position data / 查詢持倉資料
# 取得使用者永續合約持倉資訊

# GET /openApi/swap/v2/user/positions
# 頻率限制：依UID限制為每秒5次 & 依IP組合限制數量
# API KEY權限：讀取

In [ ]:
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
APIURL = "https://open-api.bingx.com"  # API 基本網址
APIKEY = ""  # 輸入你的 API Key
SECRETKEY = ""  # 輸入你的 Secret Key

In [ ]:
def demo():
    payload = {}  # GET 請求通常無主體內容
    path = '/openApi/swap/v2/user/positions'  # 查詢使用者持倉資料的 API 路徑
    method = "GET"  # HTTP 方法

    paramsMap = {
        "symbol": "BNB-USDT"  # 要查詢的交易對參數
    }

    paramsStr = parseParam(paramsMap)  # 將參數字典轉換為 URL 查詢字串並加入 timestamp
    return send_request(method, path, paramsStr, payload)  # 發送請求並回傳結果

In [ ]:
def get_sign(api_secret, payload):
    # 使用 HMAC-SHA256 演算法對 payload 進行簽名，確保請求完整性和安全
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 印出簽名以便除錯
    return signature

In [ ]:
def send_request(method, path, urlpa, payload):
    # 組合完整請求 URL，包含簽名參數
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 輸出完整請求 URL 方便查看

    headers = {
        'X-BX-APIKEY': APIKEY,  # 請求標頭放置 API Key
    }

    response = requests.request(method, url, headers=headers, data=payload)  # 發送 HTTP 請求
    return response.text  # 回傳 API 回應字串

In [ ]:
def parseParam(paramsMap):

    # 將參數字典依鍵名排序並組合成 URL 查詢字串
    sortedKeys = sorted(paramsMap)
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])

    # 加入毫秒級時間戳記，防止重放攻擊
    if paramsStr != "":
        return paramsStr + "&timestamp=" + str(int(time.time() * 1000))
    else:
        return "timestamp=" + str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())  # 執行示範函式並印出回應